<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Macro


In [ ]:
import language.experimental.macros

// Normal imports
import reflect.macros.blackbox.Context

object Step1Complete {
  def hello(): Unit = macro hello_impl

  def hello_impl(c: Context)(): c.Expr[Unit] = {
    import c.universe._
    println("Compiling!")
    reify { println("Hello World!") }
  }
     
}      

## quasiquotes 
https://docs.scala-lang.org/overviews/quasiquotes/setup.html  
可对比参考Lisp的宏

In [8]:
val universe: scala.reflect.runtime.universe.type = scala.reflect.runtime.universe
import universe._
val tree = q"i am { a quasiquote }"
val C = q"class C"
//println(showCode(C))
//println(showCode(tree))
//println(showRaw(tree))

universe: reflect.api.JavaUniverse = scala.reflect.runtime.JavaUniverse@63cd909
import universe._

tree: Tree = Apply(Select(Ident(i), am), List(Select(Ident(a), quasiquote)))
C: ClassDef = ClassDef(
  Modifiers(0L, , List()),
  C,
  List(),
  Template(
    List(Select(Ident(scala), AnyRef)),
    ValDef(Modifiers(4L, , List()), _, <notype>, <empty>),
    List(
      DefDef(
        Modifiers(0L, , List()),
        <init>,
        List(),
        List(List()),
        <type ?>,
        Block(
          List(Apply(Select(Super(This(), ), <init>), List())),
          Literal(Constant(()))
        )
      )
    )
  )
)

In [2]:
val a = q"true"
val b = true

a: Literal = Literal(Constant(true))
b: Boolean = true

In [3]:
val q"i am $what" = q"i am { a quasiquote }"

what: Tree = Select(Ident(a), quasiquote)

In [4]:
val ab = List(q"a", q"b")

ab: List[Ident] = List(Ident(a), Ident(b))

In [10]:
import scala.reflect.runtime.currentMirror
import scala.tools.reflect.ToolBox
val toolbox = currentMirror.mkToolBox()
val fab = q"f(..$ab)"
//val C = q"class C"
val tree = q"i am { a quasiquote }"
println(tree match { case q"i am { a quasiquote }" => "it worked!" })

it worked!


import scala.reflect.runtime.currentMirror

import scala.tools.reflect.ToolBox

toolbox: ToolBox[universe.type] = scala.tools.reflect.ToolBoxFactory$ToolBoxImpl@68e4d8d4
fab: Tree = Apply(Ident(f), List(Ident(a), Ident(b)))
tree: Tree = Apply(Select(Ident(i), am), List(Select(Ident(a), quasiquote)))

In [12]:
println(q"foo + bar" equalsStructure q"foo.+(bar)")

true


In [13]:
val x = q"""
         val x: List[Int] = List(1, 2) match {
           case List(a, b) => List(a + b)
         }
       """

x: ValDef = ValDef(
  Modifiers(0L, , List()),
  x,
  AppliedTypeTree(Ident(List), List(Ident(Int))),
  Match(
    Apply(Ident(List), List(Literal(Constant(1)), Literal(Constant(2)))),
    List(
      CaseDef(
        Apply(Ident(List), List(Bind(a, Ident(_)), Bind(b, Ident(_)))),
        <empty>,
        Apply(Ident(List), List(Apply(Select(Ident(a), $plus), List(Ident(b)))))
      )
    )
  )
)

In [14]:
val q"f(..$args)" = q"f(a, b)"

args: List[Tree] = List(Ident(a), Ident(b))

In [17]:
val two = 1 + 1
val four = q"$two + $two"
val ints = List(1, 2, 3)
val f123 = q"f(..$ints)"

two: Int = 2
four: Tree = Apply(
  Select(Literal(Constant(2)), $plus),
  List(Literal(Constant(2)))
)
ints: List[Int] = List(1, 2, 3)
f123: Tree = Apply(
  Ident(f),
  List(Literal(Constant(1)), Literal(Constant(2)), Literal(Constant(3)))
)

In [18]:
val q"${left: Int} + ${right: Int}" = q"2 + 2"

left: Int = 2
right: Int = 2

In [23]:
val q"..$stats" = q"(2,3)"

stats: List[Tree] = List(
  Apply(
    Select(Ident(scala), Tuple2),
    List(Literal(Constant(2)), Literal(Constant(3)))
  )
)

In [26]:
val code = q"""println("compiled and run at runtime!")"""
val compiledCode = toolbox.compile(code)
val result = compiledCode()

compiled and run at runtime!


code: Tree = Apply(
  Ident(println),
  List(Literal(Constant("compiled and run at runtime!")))
)
compiledCode: () => Any = scala.tools.reflect.ToolBoxFactory$ToolBoxImpl$ToolBoxGlobal$$Lambda$3166/2090482644@433c7cb5
result: Any = ()

In [28]:
val x = q"package mypackage { class MyClass }"
showCode(x)

x: PackageDef = PackageDef(
  Ident(mypackage),
  List(
    ClassDef(
      Modifiers(0L, , List()),
      MyClass,
      List(),
      Template(
        List(Select(Ident(scala), AnyRef)),
        ValDef(Modifiers(4L, , List()), _, <notype>, <empty>),
        List(
          DefDef(
            Modifiers(0L, , List()),
            <init>,
            List(),
            List(List()),
            <type ?>,
            Block(
              List(Apply(Select(Super(This(), ), <init>), List())),
              Literal(Constant(()))
            )
          )
        )
      )
    )
  )
)
res27_1: String = """package mypackage {
  class MyClass
}"""

In [31]:
 
  def generateCode() =
    q"package mypackage { class MyClass }"
  def saveToFile(path: String, code: Tree) = {
    val writer = new java.io.PrintWriter(path)
    try writer.write(showCode(code))
    finally writer.close()
  }
  saveToFile("myfile.scala", generateCode())
 

defined function generateCode
defined function saveToFile

In [37]:
val q"$expr.$tname" = q"a.eat"
val q"${x: scala.Symbol}" = q"'a"


cmd37.sc:3: tname is already defined as value tname
val q"$mods def $tname[..$tparams](...$paramss): $tpt = $expr" = q"""
                 ^cmd37.sc:3: expr is already defined as value expr
val q"$mods def $tname[..$tparams](...$paramss): $tpt = $expr" = q"""
                                                         ^Compilation Failed

: 

In [56]:
val q"$mods def $tname[..$tparams](...$paramss): $tpt = $expr" = q"""
private def interrupt(namePre:String, triggers:Int*)(name:String): Unit =  
println("hello scala macro")
"""

mods: Modifiers = Modifiers(4L, , List())
tname: TermName = interrupt
tparams: List[TypeDef] = List()
paramss: List[List[ValDef]] = List(
  List(
    ValDef(Modifiers(8192L, , List()), namePre, Ident(String), <empty>),
    ValDef(
      Modifiers(8192L, , List()),
      triggers,
      AppliedTypeTree(
        Select(Select(Ident(_root_), scala), <repeated>),
        List(Ident(Int))
      ),
      <empty>
    )
  ),
  List(ValDef(Modifiers(8192L, , List()), name, Ident(String), <empty>))
)
tpt: Tree = Ident(Unit)
expr: Tree = Apply(Ident(println), List(Literal(Constant("hello scala macro"))))

In [45]:
val q"..$stats0" = q"a; b; c"
val q"..$stats1" = q"(a, b, c)"
val q"..$stats2" = q"""{a
b
c}"""
val q"..$stats3" = q"""{
val a: Int = 1
val b: Int = 2
val c = 3}"""

stats0: List[Tree] = List(Ident(a), Ident(b), Ident(c))
stats1: List[Tree] = List(
  Apply(Select(Ident(scala), Tuple3), List(Ident(a), Ident(b), Ident(c)))
)
stats2: List[Tree] = List(Ident(a), Ident(b), Ident(c))
stats3: List[Tree] = List(
  ValDef(Modifiers(0L, , List()), a, Ident(Int), Literal(Constant(1))),
  ValDef(Modifiers(0L, , List()), b, Ident(Int), Literal(Constant(2))),
  ValDef(Modifiers(0L, , List()), c, <type ?>, Literal(Constant(3)))
)

In [49]:
 val extractor = pq"Foo(1, 2, 3)"
 val extractor2 = q"Foo(1, 2, 3)"
val pq"$id(..$pats)" = extractor
val pq"$id2(..$pats2)" = extractor2

extractor: Tree = Apply(
  Ident(Foo),
  List(Literal(Constant(1)), Literal(Constant(2)), Literal(Constant(3)))
)
extractor2: Tree = Apply(
  Ident(Foo),
  List(Literal(Constant(1)), Literal(Constant(2)), Literal(Constant(3)))
)
id: Tree = Ident(Foo)
pats: List[Tree] = List(
  Literal(Constant(1)),
  Literal(Constant(2)),
  Literal(Constant(3))
)
id2: Tree = Ident(Foo)
pats2: List[Tree] = List(
  Literal(Constant(1)),
  Literal(Constant(2)),
  Literal(Constant(3))
)

In [52]:
val isT =  pq"_: T"

isT: Typed = Typed(Ident(_), Ident(T))

In [53]:
val f3 = q"private implicit def f"

f3: DefDef = DefDef(
  Modifiers(532L, , List()),
  f,
  List(),
  List(),
  Select(Ident(scala), Unit),
  <empty>
)

In [ ]:
import language.experimental.macros

// Normal imports
import reflect.macros.blackbox.Context

In [ ]:
abstract class Animal {
  def name: String
}
case class Cat(name: String) extends Animal
case class Dog(name: String) extends Animal

In [ ]:
object CovarianceTest extends App {
  def printAnimalNames(animals: List[Animal]): Unit = {
    animals.foreach { animal =>
      println(animal.name)
    }
  }

  val cats: List[Cat] = List(Cat("Whiskers"), Cat("Tom"))
  val dogs: List[Dog] = List(Dog("Fido"), Dog("Rex"))

  printAnimalNames(cats)
  // Whiskers
  // Tom

  printAnimalNames(dogs)
  // Fido
  // Rex
}

In [ ]:
//CovarianceTest.printAnimalNames(CovarianceTest.cats)
 val cats: List[Cat] = List(Cat("Whiskers"), Cat("Tom"))
  val dogs: List[Dog] = List(Dog("Fido"), Dog("Rex"))
CovarianceTest.printAnimalNames(cats)
    CovarianceTest.printAnimalNames(dogs)

In [ ]:
  val HTMLFactory =
    s"""
       |<!DOCTYPE html>
       |<html>
       |  <head>
       |    <title>
       |      Regif
       |    </title>
       |    <style>
       |      div{
       |          text-align: center;
       |      }
       |
       |    </style>
       |  </head>
       |  <body>
       |  <table  align="center" class="theme-default">
       |  </table>
       |  </body>
       |</html>
       |""".stripMargin

In [ ]:
println(HTMLFactory)

In [ ]:
val htmlbody = "html5 ${body} end"
val body = "-body-"
StringContext("html5 ","end").s(body)

In [ ]:
val body = "div xxx div "
val b = s"html5 ${body} end" 
val c = new StringContext(htmlbody).s(htmlbody)
val d = StringContext("html5 ${body} end")
 
//val bf = "html5 %s end".format(body)

In [ ]:
class T1 extends Component{
    val f = UInt"11000011"
}
showRtl(new T1)

In [3]:
import scala.reflect.macros.whitebox.Context
import scala.language.experimental.macros
import scala.annotation.StaticAnnotation
object helloMacro {
  def impl(c: Context)(annottees: c.Expr[Any]*): c.Expr[Any] = {
    import c.universe._
    val result = {
      annottees.map(_.tree).toList match {
        case ModuleDef(mods, name, Template(parents, self, body)) :: Nil =>
          val helloMethod = DefDef(NoMods, TermName("hello"), List(), List(List()), TypeTree(), Literal(Constant("hello")))
          ModuleDef(mods, name, Template(parents, self, body :+ helloMethod))
      }
    }
    c.Expr[Any](result)
  }
}

import scala.reflect.macros.whitebox.Context

import scala.language.experimental.macros

import scala.annotation.StaticAnnotation

defined object helloMacro

In [4]:
import scala.reflect.macros.blackbox
object dataMacro {
  def impl(c: blackbox.Context)(annottees: c.Expr[Any]*): c.Expr[Any] = {
    import c.universe._

    val result = {
      annottees.map(_.tree).toList match {
        case q"""
              class $name {
                ..$vars
              }
              """ :: Nil =>

          // Generate the Getter and Setter from VarDefs
          val beanMethods = vars.collect {
            case q"$mods var $name: $tpt = $expr" =>
              val getName = TermName("get" + name.encodedName.toString.capitalize)
              val setName = TermName("set" + name.encodedName.toString.capitalize)
              println(getName)
              val ident = Ident(name)
              List (
                q"def $getName: $tpt = $ident",
                q"def $setName(paramX: $tpt): Unit = { $ident = paramX }"
              )
          }.flatten

          // Insert the generated Getter and Setter
          q"""
             class $name {
               ..$vars
               ..$beanMethods
             }
           """
        case _ =>
          throw new Exception("Macro Error")
      }
    }
    c.Expr[Any](result)
  }
}


import scala.reflect.macros.blackbox

defined object dataMacro

In [4]:
class data extends StaticAnnotation {
  def macroTransform(annottees: Any*): Any = macro dataMacro.impl
}

cmd4.sc:2: macro implementation reference has wrong shape. required:
macro [<static object>].<method name>[[<type args>]] or
macro [<macro bundle>].<method name>[[<type args>]]
  def macroTransform(annottees: Any*): Any = macro dataMacro.impl
                                                             ^Compilation Failed

: 

In [ ]:
      错了 